In [1]:
import numpy as np
import pandas as pd
import spacy
import matplotlib.pyplot as plt 
from itertools import chain
from wordcloud import WordCloud, STOPWORDS 
from nltk.stem import PorterStemmer
import re

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

%run ./jlu_preprocessing.ipynb
%run ./spk_preprocessing.ipynb

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# data = pd.read_excel(r'C:/Users/Checkout/Desktop/SJSU/sem1/257-ML/Project/global_warming_tweets.xls')
data = pd.read_csv(r'global_warming_tweets.csv', engine='python') #encoding = "cp1252"
data.head()

,tweet,existence,existence.confidence
0,Global warming report urges governments to act...,Yes,1.0000
1,Fighting poverty and global warming in Africa ...,Yes,1.0000
2,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
3,Carbon offsets: How a Vatican forest failed to...,Yes,1.0000
4,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087


In [4]:
data['existence'] = data['existence'].fillna('neutral')

In [5]:
data[data['existence']=="No"].tweet

17      Wait here's an idea: it's natural climate chan...
31      @New_federalists  i have it on good auth tht g...
32      Illegal war and the myth of global warming|My ...
33      the scientific community was scamed by global ...
35      40 degrees in NYC. please urinate on next libe...
                              ...                        
781     Don Blankenship Calls Efforts On Mine Safety R...
1006    Don't be swayed by distraction of climate chan...
1058    Who Cares About Global Warming?: by Jackie Gin...
1071    Don Blankenship Called Safety Regulators "as S...
1072    Coal CEO Calls Mine Safety Rules 'As Silly As ...
Name: tweet, Length: 61, dtype: object

In [6]:
print(data['existence'].value_counts())

Y          2554
neutral    1865
N          1053
Yes         557
No           61
Name: existence, dtype: int64


In [7]:
data['word_count'] = data['tweet'].apply(lambda x: len(x.split(" ")))
data = data.drop_duplicates()
data = data.dropna()
data.loc[data['existence'] == 'Y','existence'] = "Yes"
data.loc[data['existence'] == 'N','existence'] = "No"
print(data.shape)
data.dropna()
data.loc[data['existence'] == np.nan,'existence'] = "No"
print(data['existence'].value_counts())
print(data.shape)
data.head()

(5960, 4)
Yes        3029
neutral    1826
No         1105
Name: existence, dtype: int64
(5960, 4)


,tweet,existence,existence.confidence,word_count
0,Global warming report urges governments to act...,Yes,1.0000,18
1,Fighting poverty and global warming in Africa ...,Yes,1.0000,8
2,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786,12
3,Carbon offsets: How a Vatican forest failed to...,Yes,1.0000,12
4,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087,11


In [8]:
data['tweet']

0       Global warming report urges governments to act...
1       Fighting poverty and global warming in Africa ...
2       Carbon offsets: How a Vatican forest failed to...
3       Carbon offsets: How a Vatican forest failed to...
4       URUGUAY: Tools Needed for Those Most Vulnerabl...
                              ...                        
6085    @bloodless_coup "The phrase 'global warming' s...
6086    Virginia to Investigate Global Warming Scienti...
6087    Global warming you tube parody you will enjoy ...
6088    One-Eyed Golfer: Don't dare tell me about glob...
6089    man made global warming a hair brained theory ...
Name: tweet, Length: 5960, dtype: object

In [9]:
tweets = data["tweet"]
# tweets = tweets.drop_duplicates()
tweets

0       Global warming report urges governments to act...
1       Fighting poverty and global warming in Africa ...
2       Carbon offsets: How a Vatican forest failed to...
3       Carbon offsets: How a Vatican forest failed to...
4       URUGUAY: Tools Needed for Those Most Vulnerabl...
                              ...                        
6085    @bloodless_coup "The phrase 'global warming' s...
6086    Virginia to Investigate Global Warming Scienti...
6087    Global warming you tube parody you will enjoy ...
6088    One-Eyed Golfer: Don't dare tell me about glob...
6089    man made global warming a hair brained theory ...
Name: tweet, Length: 5960, dtype: object

In [10]:
# tweets[:100]

In [11]:
# python -m spacy download en_core_web_sm
preprocessed_tweets,indices = preprocess_tweets(tweets)

{'belgium', 'increase', 'global', 'face', 'warming', 'report', 'hunger', 'urge', 'government', 'world'} 
---

{'poverty', 'global', 'fight', 'africa', 'warming'} 
---

{'offset', 'carbon', 'global', 'warming', 'vatican', 'reduce', 'fail', 'forest'} 
---

{'offset', 'carbon', 'global', 'warming', 'vatican', 'reduce', 'fail', 'forest'} 
---

{'uruguay', 'tool', 'need', 'vulnerable', 'climate', 'change'} 
---

{'saltiness', 'cycle', 'water', 'intensify', 'global', 'warming', 'ocean', 'show'} 
---

{'message', 'denier', 'global', 'look', 'warming', 'evidence', 'doubter'} 
---

{'stay', 'bird', 'strategy', 'migratory', 'climate', 'change', 'home'} 
---

{'southe', 'compete', 'high', 'southern', 'water', 'limpopo', 'africa', 'temperature', 'bring', 'climate', 'change'} 
---

{'wheat', 'rice', 'water', 'global', 'warming', 'production', 'impact', 'scarcity'} 
---

{'analysis', 'global', 'warming', 'previously', 'natural', 'suggest', 'contribute', 'preliminary'} 
---

{'population', 'climate',

{'cause', 'cancer', 'global', 'mental', 'warming', 'say', 'report', 'illness', 'government'} 
---

{'heat', 'increase', 'climate', 'flood', 'wave', 'change'} 
---

{'plant', 'global', 'effective', 'remain', 'warming', 'tackle'} 
---

{'get', 'class', 'global', 'student', 'heated', 'climate', 'deni', 'fault', 'discussion', 'change'} 
---

{'colorado', 'kill', 'global', 'warming', 'forest'} 
---

{'display', 'confirm', 'occur', 'environmental', 'climate', 'change'} 
---

{'europe', 'global', 'warming', 'place', 'true', 'foll', 'prediction', 'prove', 'disappear'} 
---

{'indigenous', 'combat', 'people', 'climate', 'change', 'lesson'} 
---

{'strong', 'action', 'bill', 'live', 'support', 'climate', 'change'} 
---

{'indigenous', 'fight', 'climate', 'tradition', 'change'} 
---

{'green', 'go', 'hope', 'solution', 'false', 'heather', 'real', 'roger', 'wrong', 'climate', 'change', 'james', 'hansen'} 
---

{'absorb', 'carbon', 'activist', 'climate', 'forest', 'change'} 
---

{'absorb', 'carbon

{'health', 'nation', 'feel', 'industrial', 'report', 'impact', 'climate', 'change'} 
---

{'credit', 'green', 'step', 'carbon', 'mention', 'living', 'daunting', 'gecko', 'climate', 'change'} 
---

{'biofuel', 'credit', 'green', 'step', 'carbon', 'mention', 'living', 'daunting', 'climate', 'change'} 
---

{'cost', 'reuter', 'raise', 'climate', 'allergy', 'change'} 
---

{'asilomar', 'curable', 'treat', 'disease', 'later', 'international', 'bioethicist', 'expert', 'climate', 'change'} 
---

{'asilomar', 'curable', 'treat', 'disease', 'later', 'international', 'bioethicist', 'expert', 'climate', 'change'} 
---

{'coal', 'safety', 'rule', 'crazy', 'global', 'warming', 'call', 'silly'} 
---

{'lead', 'fight', 'climate', 'military', 'change'} 
---

{'buyer', 'lead', 'fight', 'climate', 'fuel', 'government', 'large', 'military', 'change'} 
---

{'biofuel', 'credit', 'green', 'step', 'carbon', 'mention', 'living', 'daunting', 'climate', 'change'} 
---

{'cause', 'goldberg', 'global', 'main', '

{'oslo', 'cause', 'volcano', 'jonah', 'goldberg', 'iceland', 'global', 'warming', 'reuters', 'thaw', 'eruption'} 
---

{'conversation', 'indigenous', 'voice', 'bring', 'climate', 'change'} 
---

{'volcano', 'awaken', 'volcanic', 'icelandic', 'change', 'climate', 'thaw', 'activity'} 
---

{'south', 'nikolas', 'amazon', 'kozloff', 'rain', 'affect', 'entire', 'climate', 'change'} 
---

{'place', 'america', 'photo', 'climate', 'change'} 
---

{'bury', 'volcano', 'iceland', 'global', 'warming', 'hmmm', 'glacier', 'lede', 'melt'} 
---

{'sure', 'volcano', 'scientist', 'lead', 'consensus', 'crisis', 'warn', 'climate', 'change', 'melt'} 
---

{'right', 'climate', 'mother', 'earth', 'world', 'people', 'wish', 'conference', 'change', 'cochabamba'} 
---

{'high', 'make', 'feel', 'near', 'record', 'seasonal', 'climate', 'allergy', 'change', 'bad', 'spring'} 
---

{'high', 'make', 'near', 'record', 'seasonal', 'climate', 'allergy', 'change', 'bad', 'spring'} 
---

{'disrupt', 'volcanic', 'icelandic

{'call', 'document', 'climate', 'change', 'tackle'} 
---

{'usgs', 'go', 'park', 'global', 'warming', 'namesake', 'report', 'glacier'} 
---

{'heat', 'feel', 'come', 'jackson', 'institute', 'climate', 'erwin', 'change'} 
---

{'democracy', 'college', 'lead', 'mobilise', 'schumacher', 'climate', 'change', 'tackle'} 
---

{'cost', 'exclusive', 'raise', 'climate', 'allergy', 'change'} 
---

{'health', 'environmental', 'study', 'receive', 'impact', 'harvard', 'climate', 'change', 'million'} 
---

{'today', 'lead', 'write', 'hill', 'climate', 'change', 'kerry'} 
---

{'cost', 'exclusive', 'raise', 'climate', 'allergy', 'change'} 
---

{'fortunegreen', 'chouinard', 'patagonia', 'global', 'warming', 'industry', 'place', 'yvon', 'clothing', 'start'} 
---

{'fight', 'filmmaker', 'travel', 'change', 'people', 'climate', 'globe', 'find'} 
---

{'fight', 'filmmaker', 'travel', 'change', 'people', 'climate', 'globe', 'find'} 
---

{'spur', 'effort', 'energy', 'future', 'harm', 'fight', 'global', 'w

...

{'ensue', 'global', 'warming', 'control', 'planet', 'influence', 'winter', 'nuclear'} 
---

{'graph', 'tlot', 'attack', 'hhrs', 'teaparty', 'global', 'tcot', 'warming', 'study'} 
---

{'denier', 'green', 'love', 'climate', 'change'} 
---

{'tlot', 'analyst', 'laughable', 'global', 'tcot', 'state', 'heartland', 'warming', 'report', 'dept'} 
---

{'ppls', 'watch', 'live', 'world', 'conf', 'support', 'crank', 'bolivia', 'climate', 'change', 'hand'} 
---

{'author', 'global', 'hour', 'dennis', 'warming', 'great', 'blunder', 'spencer', 'miller'} 
---

{'heat', 'cool', 'clean', 'issue', 'publish', 'handy', 'planet', 'guide', 'climate', 'washington', 'change'} 
---

{'life', 'flickr', 'second', 'photo', 'bolivia', 'climate', 'change', 'summit'} 
---

{'volcano', 'news', 'math', 'time', 'icleand', 'global', 'destroy', 'science', 'warming', 'say', 'release', 'earth'} 
---

{'wellduh', 'global', 'warming', 'control', 'planet', 'recycle', 'light', 'climate', 'change', 'believe'} 
---

{'tomorrow'

{'geologist', 'cincinnati', 'pattern', 'lowell', 'study', 'historic', 'climate', 'change', 'university'} 
---

{'hopkins', 'mark', 'problem', 'global', 'warming', 'real', 'hoax', 'prove'} 
---

{'slade', 'gile', 'mccarthyism', 'palin', 'africa', 'strange', 'think', 'sarah', 'climate', 'change', 'countr'} 
---

{'prepare', 'compromise', 'introduce', 'month', 'climate', 'bill', 'sweeping', 'reuters', 'senator', 'washington', 'change'} 
---

{'market', 'global', 'warming', 'science', 'christian', 'proposal', 'love', 'debate', 'monitor', 'libertarian'} 
---

{'claim', 'cause', 'begin', 'global', 'warming', 'story', 'government', 'promise', 'climate', 'change'} 
---

{'focus', 'dana', 'ridge', 'mass', 'climate', 'environmental', 'change', 'conference', 'representativ', 'center'} 
---

{'focus', 'dana', 'ridge', 'mass', 'climate', 'environmental', 'change', 'conference', 'representativ', 'center'} 
---

{'safe', 'folk', 'global', 'warming', 'promise', 'offshore', 'sweat', 'drilling'} 
---

{

{'volcano', 'cover', 'answer', 'climate', 'change', 'question'} 
---

{'volcano', 'cover', 'answer', 'suvs', 'think', 'climate', 'change', 'question'} 
---

{'cause', 'flow', 'sea', 'earthquake', 'global', 'suuuuure', 'warming', 'warn', 'glacier', 'land'} 
---

{'neutral', 'corp', 'news', 'carbon', 'aim', 'promote', 'climate', 'change'} 
---

{'gaetz', 'fear', 'times', 'colonistapril', 'cost', 'global', 'warming', 'barry', 'climategat', 'skeptic', 'remedy'} 
---

{'talk', 'film', 'april', 'global', 'scientifique', 'warming', 'presentation', 'topi', 'present'} 
---

{'waiting', 'kiddo', 'newyork', 'florida', 'nice', 'global', 'miss', 'warming', 'win', 'think'} 
---

{'waiting', 'kiddo', 'newyork', 'florida', 'nice', 'global', 'miss', 'warming', 'win', 'think'} 
---

{'waiting', 'kiddo', 'newyork', 'florida', 'nice', 'global', 'miss', 'warming', 'win', 'think'} 
---

{'waiting', 'kiddo', 'newyork', 'florida', 'nice', 'global', 'miss', 'warming', 'win', 'think'} 
---

{'waiting', 'kiddo',

{'get', 'news', 'brkg', 'scientist', 'sue', 'iceland', 'volcanic', 'eruption', 'rehabilitate', 'announce', 'climate', 'change', 'panel'} 
---

{'heatwave', 'april', 'global', 'warming', 'record', 'set', 'prove'} 
---

{'oslo', 'cause', 'cap', 'global', 'turn', 'warming', 'climat', 'come', 'myth', 'reuters', 'ashe', 'thaw', 'decade'} 
---

{'time', 'global', 'short', 'warming', 'pollution', 'run', 'world', 'alarmism'} 
---

{'treehugger', 'effect', 'earthquake', 'climate', 'environment', 'political', 'economic', 'change'} 
---

{'changin', 'talk', 'cool', 'time', 'global', 'warming', 'bill', 'mckibben'} 
---

{'adapt', 'hominid', 'past', 'climate', 'change'} 
---

{'news', 'history', 'site', 'environment', 'earth', 'great', 'aspect', 'natural', 'climate', 'break', 'change'} 
---

{'carter', 'volcano', 'beck', 'news', 'glenn', 'smoke', 'algore', 'jimmy', 'global', 'tcot', 'warming', 'send', 'nowplaying', 'msnbc', 'signal', 'gore'} 
---

{'carter', 'volcano', 'beck', 'news', 'glenn', 'smo

{'paul', 'make', 'good', 'krugman', 'love', 'economic', 'climate', 'jasencomstock', 'change', 'primer'} 
---

{'long', 'exploration', 'horizon', 'post', 'climate', 'change'} 
---

{'stop', 'join', 'global', 'warming', 'kill', 'valero', 'want'} 
---

{'introduce', 'track', 'month', 'global', 'warming', 'bill', 'sweeping', 'senate', 'climate', 'flop', 'change'} 
---

{'violence', 'elect', 'right', 'ecojustice', 'daily', 'system', 'climate', 'change'} 
---

{'global', 'warming', 'answer', 'think', 'hoax', 'question', 'find'} 
---

{'effort', 'global', 'international', 'warming', 'negotia', 'bonn', 'agree', 'continue'} 
---

{'process', 'time', 'million', 'year', 'call', 'alaska', 'thirty', 'climate', 'vanish', 'change', 'tree', 'palm'} 
---

{'scientist', 'global', 'warm', 'challenge', 'intl', 'conference'} 
---

{'telegraph', 'today', 'refute', 'second', 'global', 'daily', 'warming', 'study', 'meat', 'declare', 'oppose', 'view'} 
---

{'consistent', 'fanatic', 'environmentalist', 'global

## Modeling

In [12]:
new_data = data.iloc[indices]

In [13]:
prepr_tweets = [" ".join(each) for each in preprocessed_tweets]
new_data['cleaned_tweet'] = prepr_tweets
new_data.head()

,tweet,existence,existence.confidence,word_count,cleaned_tweet
0,Global warming report urges governments to act...,Yes,1.0000,18,belgium increase global face warming report hu...
1,Fighting poverty and global warming in Africa ...,Yes,1.0000,8,poverty global fight africa warming
2,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786,12,offset carbon global warming vatican reduce fa...
3,Carbon offsets: How a Vatican forest failed to...,Yes,1.0000,12,offset carbon global warming vatican reduce fa...
4,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087,11,uruguay tool need vulnerable climate change


In [14]:
len(prepr_tweets), data.shape, new_data.shape

(5544, (5960, 4), (5544, 5))

In [15]:
# TF IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import collections, numpy

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(new_data['cleaned_tweet'].astype('U'))

tf = TfidfVectorizer()
text_tf = tf.fit_transform(new_data['cleaned_tweet'].astype('U'))
# print(text_tf)

In [16]:
new_data.isna().sum()

tweet                   0
existence               0
existence.confidence    0
word_count              0
cleaned_tweet           0
dtype: int64

In [17]:
# compute similarity using cosine similarity
cos_sim=cosine_similarity(text_tf, text_tf)
print(cos_sim)

[[1.         0.03273816 0.02203327 ... 0.02147607 0.02021837 0.0164164 ]
 [0.03273816 1.         0.03854919 ... 0.03757431 0.03537386 0.02872197]
 [0.02203327 0.03854919 1.         ... 0.02528807 0.02380713 0.01933031]
 ...
 [0.02147607 0.03757431 0.02528807 ... 1.         0.02320507 0.01884147]
 [0.02021837 0.03537386 0.02380713 ... 0.02320507 1.         0.01773806]
 [0.0164164  0.02872197 0.01933031 ... 0.01884147 0.01773806 1.        ]]


In [18]:
# splitting data 

X_train, X_test, y_train, y_test = train_test_split(cos_sim, new_data['existence'], test_size=0.2, random_state=33)
print(" Test Data Shape:", X_test.shape)
print(" Train Data Shape:",X_train.shape)

 Test Data Shape: (1109, 5544)
 Train Data Shape: (4435, 5544)


In [19]:
pos = (y_test == 'Yes').sum()
neg = (y_test == 'No').sum()
postrain = (y_train == 'Yes').sum()
negtrain = (y_train == 'No').sum()
total = pos + neg
print(" Test Data Positive Sentiments :", pos)
print(" Test Data Negative Sentiments :",neg)
print(" Train Data Positive Sentiments :", postrain)
print(" Train Data Positive Sentiments :",negtrain)
new_data['existence'].value_counts()

 Test Data Positive Sentiments : 588
 Test Data Negative Sentiments : 198
 Train Data Positive Sentiments : 2261
 Train Data Positive Sentiments : 817


Yes        2849
neutral    1680
No         1015
Name: existence, dtype: int64

In [20]:
# perform algoritma KNN
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import precision_score, auc,recall_score, f1_score,roc_curve,confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC


In [21]:
clfs = [
    svm.SVC(kernel='linear').fit(X_train, y_train),
#     LinearSVC().fit(X_train, y_train),
    DecisionTreeClassifier().fit(X_train, y_train),
    KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
]

In [22]:
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)

In [ ]:
row_index = 0
for clf in clfs:
    print('===============================================\n')
    print("**********{}***********".format(clf.__class__.__name__))
    predicted = clf.predict(X_test)
#     tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
    MLA_name = clf.__class__.__name__
    MLA_compare.loc[row_index,'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Train Accuracy'] = round(clf.score(X_train, y_train), 2)
    MLA_compare.loc[row_index, 'MLA Test Accuracy'] = round( accuracy_score(y_test,predicted), 2)
    MLA_compare.loc[row_index, 'MLA Precision'] = round( precision_score(y_test,predicted, average="macro"), 2)
    MLA_compare.loc[row_index, 'MLA Recall'] = round( recall_score(y_test,predicted, average="macro"), 2)
    MLA_compare.loc[row_index, 'MLA F1 Score'] = round( f1_score(y_test,predicted, average="macro"), 2)
    MLA_compare.loc[row_index, 'error_rate'] = round( 1-accuracy_score(y_test,predicted), 2)
    MLA_compare.loc[row_index, 'cross val score'] = cross_val_score(clf, cos_sim,new_data['existence'], cv=10).mean()
    row_index+=1




**********SVC***********

**********DecisionTreeClassifier***********


In [ ]:
MLA_compare.sort_values(by = ['MLA Test Accuracy'], ascending = False, inplace = True)
MLA_compare

## Random Under Sampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# splitting data 

X_train, X_test, y_train, y_test = train_test_split(cos_sim, new_data['existence'], test_size=0.2, random_state=33)
print(" Test Data Shape:", X_test.shape)
print(" Train Data Shape:",X_train.shape)

In [ ]:
X_train,y_train = RandomUnderSampler(random_state = 21).fit_resample(X_train,y_train)
# X_train,y_train = RandomOverSampler(random_state = 21).fit_resample(X_train,y_train)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
under_MLA_columns = []
under_MLA_compare = pd.DataFrame(columns = under_MLA_columns)

In [ ]:
under_clfs = [
    svm.SVC(kernel='linear').fit(X_train, y_train),
#     LinearSVC(C=0.0001).fit(X_train, y_train),
    DecisionTreeClassifier().fit(X_train, y_train),
    KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
]

In [ ]:
row_index = 0
for clf in under_clfs:
    print('===============================================\n')
    print("**********{}***********".format(clf.__class__.__name__))
    predicted = clf.predict(X_test)
#     tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
    under_MLA_name = clf.__class__.__name__
    under_MLA_compare.loc[row_index,'MLA Name'] = under_MLA_name
    under_MLA_compare.loc[row_index, 'MLA Train Accuracy'] = round(clf.score(X_train, y_train), 2)
    under_MLA_compare.loc[row_index, 'MLA Test Accuracy'] = round( accuracy_score(y_test,predicted), 2)
    under_MLA_compare.loc[row_index, 'MLA Precision'] = round( precision_score(y_test,predicted, average="macro"), 2)
    under_MLA_compare.loc[row_index, 'MLA Recall'] = round( recall_score(y_test,predicted, average="macro"), 2)
    under_MLA_compare.loc[row_index, 'MLA F1 Score'] = round( f1_score(y_test,predicted, average="macro"), 2)
    under_MLA_compare.loc[row_index, 'error_rate'] = round( 1-accuracy_score(y_test,predicted), 2)
    under_MLA_compare.loc[row_index, 'cross val score'] = cross_val_score(clf, cos_sim,new_data['existence'], cv=10).mean()
    row_index+=1


In [ ]:
under_MLA_compare.sort_values(by = ['MLA Test Accuracy'], ascending = False, inplace = True)
under_MLA_compare

## Random Over Sampling

In [ ]:
# splitting data 

X_train, X_test, y_train, y_test = train_test_split(cos_sim, new_data['existence'], test_size=0.2, random_state=33)
print(" Test Data Shape:", X_test.shape)
print(" Train Data Shape:",X_train.shape)

In [ ]:
# Xtrain,ytrain = RandomUnderSampler(random_state = 21).fit_resample(X_train,y_train)
X_train,y_train = RandomOverSampler(random_state = 21).fit_resample(X_train,y_train)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
over_MLA_columns = []
over_MLA_compare = pd.DataFrame(columns = over_MLA_columns)

In [ ]:
over_clfs = [
    svm.SVC(kernel='linear').fit(X_train, y_train),
#     LinearSVC(C=0.0001).fit(X_train, y_train),

    DecisionTreeClassifier().fit(X_train, y_train),
    KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
]

In [ ]:
row_index = 0
for clf in over_clfs:
    print('===============================================\n')
    print("**********{}***********".format(clf.__class__.__name__))
    predicted = clf.predict(X_test)
#     tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
    over_MLA_name = clf.__class__.__name__
    over_MLA_compare.loc[row_index,'MLA Name'] = over_MLA_name
    over_MLA_compare.loc[row_index, 'MLA Train Accuracy'] = round(clf.score(X_train, y_train), 2)
    over_MLA_compare.loc[row_index, 'MLA Test Accuracy'] = round( accuracy_score(y_test,predicted), 2)
    over_MLA_compare.loc[row_index, 'MLA Precision'] = round( precision_score(y_test,predicted, average="macro"), 2)
    over_MLA_compare.loc[row_index, 'MLA Recall'] = round( recall_score(y_test,predicted, average="macro"), 2)
    over_MLA_compare.loc[row_index, 'MLA F1 Score'] = round( f1_score(y_test,predicted, average="macro"), 2)
    over_MLA_compare.loc[row_index, 'error_rate'] = round( 1-accuracy_score(y_test,predicted), 2)
    over_MLA_compare.loc[row_index, 'cross val score'] = cross_val_score(clf, cos_sim,new_data['existence'], cv=10).mean()
    row_index+=1


## SMOTE

In [ ]:
# splitting data 
X_train, X_test, y_train, y_test = train_test_split(cos_sim, new_data['existence'], test_size=0.2, random_state=33)
print(" Test Data Shape:", X_test.shape)
print(" Train Data Shape:",X_train.shape)

In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE(k_neighbors=5)
X_train,y_train = oversample.fit_resample(X_train,y_train)
X_train.shape,y_train.shape

In [ ]:
SMOTE_MLA_columns = []
SMOTE_MLA_compare = pd.DataFrame(columns = SMOTE_MLA_columns)

In [ ]:
smote_clfs = [
    svm.SVC(kernel='linear').fit(X_train, y_train),
#     LinearSVC(C=0.0001).fit(X_train, y_train),

    DecisionTreeClassifier().fit(X_train, y_train),
    KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
]

In [ ]:
row_index = 0
for clf in smote_clfs:
    print('===============================================\n')
    print("**********{}***********".format(clf.__class__.__name__))
    predicted = clf.predict(X_test)
#     tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
    SMOTE_MLA_name = clf.__class__.__name__
    SMOTE_MLA_compare.loc[row_index,'MLA Name'] = SMOTE_MLA_name
    SMOTE_MLA_compare.loc[row_index, 'MLA Train Accuracy'] = round(clf.score(X_train, y_train), 2)
    SMOTE_MLA_compare.loc[row_index, 'MLA Test Accuracy'] = round( accuracy_score(y_test,predicted), 2)
    SMOTE_MLA_compare.loc[row_index, 'MLA Precision'] = round( precision_score(y_test,predicted, average="macro"), 2)
    SMOTE_MLA_compare.loc[row_index, 'MLA Recall'] = round( recall_score(y_test,predicted, average="macro"), 2)
    SMOTE_MLA_compare.loc[row_index, 'MLA F1 Score'] = round( f1_score(y_test,predicted, average="macro"), 2)
    SMOTE_MLA_compare.loc[row_index, 'error_rate'] = round( 1-accuracy_score(y_test,predicted), 2)
    SMOTE_MLA_compare.loc[row_index, 'cross val score'] = cross_val_score(clf, cos_sim,new_data['existence'], cv=10).mean()
    row_index+=1


In [ ]:
MLA_compare

In [ ]:
under_MLA_compare

In [ ]:
over_MLA_compare

In [ ]:
SMOTE_MLA_compare

In [ ]:
# import pickle
# # pickling the vectorizer
# pickle.dump(tf, open('vectorizer', 'wb'))
# # pickling the model
# pickle.dump(clfs[0], open('svm_classifier_89', 'wb'))